konlpy 설치
- 환경변수 편집 jvm.dll 경로 확인 : C:\Program Files\Java\jdk1.8.0_261\jre\bin\server

https://data-scientist-brian-kim.tistory.com/79


In [5]:
import konlpy

In [7]:
import pandas as pd

train_df = pd.read_csv('./dataset/ratings_train.txt',sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [10]:
train_df.loc[4,:].document


'사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'

In [12]:
train_df.label.value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [16]:
# 데이터 전처리
import re
import pandas as pd

train_df = train_df.fillna(' ')
# 숫자를 공백으로 처리
train_df['document'] = train_df['document'].apply(lambda x:re.sub(r"\d+"," ",x))
test_df = pd.read_csv('./dataset/ratings_test.txt',sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x:re.sub(r"\d+"," ",x))
# id 삭제
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id',axis=1, inplace=True)

In [28]:
from konlpy.tag import Okt
okt = Okt()
def okt_tokenizer(text):
    # 입력 인자로 들어온 text를 형태소 단어로 토큰화하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df =3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [ ]:
# 과제 lr으로 모델링 및 하이퍼 파라미터 튜닝후 테스트 데이터로 평가하세요

In [32]:
import warnings
warnings.filterwarnings('ignore')
# 로지스틱 회귀를 통한 감성 분석 분류 실행
lg_clf = LogisticRegression(random_state=0)
# 파라미터 C 최적화
params = {'C':[1,3.5,4.5,5.5,10]}
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv =3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 3.5} 0.8593


In [33]:
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도:', accuracy_score(test_df['label'],preds))

Logistic Regression 정확도: 0.86186
